In [1]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [7]:
#here we are installing google collab using pip
!pip install google-colab --user

  Using cached google_colab-1.0.0-py2.py3-none-any.whl
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Using cached requests-2.21.0-py2.py3-none-any.whl (57 kB)
  Using cached six-1.12.0-py2.py3-none-any.whl (10 kB)
  Using cached ipykernel-4.6.1-py3-none-any.whl (104 kB)
  Using cached pandas-0.24.2.tar.gz (11.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached ipython-5.5.0-py3-none-any.whl (758 kB)
  Using cached portpicker-1.2.0-py3-none-any.whl
  Using cached prompt_toolkit-1.0.18-py3-none-any.whl (245 kB)
  Using cached simplegeneric-0.8.1-py3-none-any.whl
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached urllib3-1.24.3-py2.py3-none-any.whl (118 kB)
  Using cached idna-2.8-py2.py3-none-any.whl (58 kB)
  Using cached jupyter_client-8.0.1-py3-none-any.whl (103 kB)
  Using cached jupyter_client-8.0.0-py3-none-any.w

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [843 lines of output]
  C:\Users\PARTI\AppData\Local\Temp\pip-install-v2kmnclb\pandas_c63fca9f4b1d438ead8ca44b988f4ed7\setup.py:50: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
    _CYTHON_INSTALLED = ver >= LooseVersion(min_cython_ver)
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-39
  creating build\lib.win-amd64-cpython-39\pandas
  copying pandas\conftest.py -> build\lib.win-amd64-cpython-39\pandas
  copying pandas\testing.py -> build\lib.win-amd64-cpython-39\pandas
  copying pandas\_version.py -> build\lib.win-amd64-cpython-39\pandas
  copying pandas\__init__.py -> build\lib.win-amd64-cpython-39\pandas
  creating build\lib.win-amd64-cpython-39\pandas\api
  copying pandas\api\__init__.py -> build\lib.win-amd64-cpython-39\pandas\api
  creating build\lib.wi

In [8]:
from google.colab import files #here you can load in your own file not necessary for project
path_to_file = list(files.upload().keys())[0]

ModuleNotFoundError: No module named 'google.colab'

In [9]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [10]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [11]:
#Here we are encoding the text
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
    return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [12]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [13]:
def int_to_text(ints):
    try:
        ints = ints.numpy()
    except:
        pass
    return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [14]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [15]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [16]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [17]:
for x, y in dataset.take(2):
    print("\n\nEXAMPLE\n")
    print("INPUT")
    print(int_to_text(x))
    print("\nOUTPUT")
    print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [18]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [19]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [20]:
for input_example_batch, target_example_batch in data.take(1):
    example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [21]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-1.9808575e-03 -4.3204259e-03  1.7222618e-03 ...  4.6632430e-03
   -1.8540709e-03 -2.3890126e-03]
  [ 1.6118970e-03 -6.8970993e-03  3.1020672e-03 ...  3.6238567e-03
   -1.0915014e-03  1.6150915e-03]
  [ 1.8035050e-03 -1.0871902e-02  2.6056229e-03 ... -9.9308067e-04
    8.0298465e-03 -7.8410108e-04]
  ...
  [-6.1958656e-03 -3.7345958e-03  1.6518899e-03 ... -7.3197791e-03
    2.3598461e-03  1.7876073e-03]
  [-8.3279330e-03 -7.7637741e-03  3.4491462e-03 ... -4.5399987e-03
    6.7595867e-03  2.1478275e-03]
  [-2.5914973e-03 -3.3854784e-03  4.0330938e-03 ... -7.2167404e-03
    2.2293371e-03  2.5037967e-03]]

 [[ 8.4803905e-04 -4.9234782e-03 -2.7070544e-04 ... -2.8395364e-03
    8.7002050e-03 -1.2024862e-03]
  [-8.8309282e-03  7.2123355e-04 -1.6743923e-04 ... -2.0345715e-03
    7.2113024e-03  2.4760230e-03]
  [-6.1548525e-04  1.5125470e-03 -3.8205185e-03 ...  2.4578179e-04
    6.8340818e-03 -1.5390028e-03]
  ...
  [-8.7047461e-03 -7.2184103e-03  1.1741356e-03 ... -1.2355748e

In [22]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00198086 -0.00432043  0.00172226 ...  0.00466324 -0.00185407
  -0.00238901]
 [ 0.0016119  -0.0068971   0.00310207 ...  0.00362386 -0.0010915
   0.00161509]
 [ 0.00180351 -0.0108719   0.00260562 ... -0.00099308  0.00802985
  -0.0007841 ]
 ...
 [-0.00619587 -0.0037346   0.00165189 ... -0.00731978  0.00235985
   0.00178761]
 [-0.00832793 -0.00776377  0.00344915 ... -0.00454     0.00675959
   0.00214783]
 [-0.0025915  -0.00338548  0.00403309 ... -0.00721674  0.00222934
   0.0025038 ]], shape=(100, 65), dtype=float32)


In [23]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-1.9808575e-03 -4.3204259e-03  1.7222618e-03  2.4003384e-03
 -1.9519140e-03  2.8603124e-03 -7.2498657e-03 -7.8751647e-04
  8.9064881e-04  1.8253585e-04  1.8289178e-03 -3.3326591e-03
 -3.8343712e-03  1.1604498e-03 -9.6757419e-04 -1.0291586e-03
 -1.7972480e-03  3.0618368e-03  1.5016374e-03 -3.1765900e-03
  2.9757898e-03  6.5395981e-04  3.7483394e-03  2.9272805e-03
  2.3671412e-03  1.2113091e-03 -5.1593236e-03 -4.2947507e-03
  2.6219653e-03  1.2084681e-03 -3.0014047e-03  1.9984310e-03
 -1.0578330e-03  1.0162295e-03  4.5720572e-03  5.2641798e-04
  3.1970167e-03 -2.2954149e-03 -9.0908911e-03  3.9375061e-04
  5.6747263e-03 -2.6674315e-03 -3.1157737e-03 -9.1613699e-03
  1.0837298e-03  2.9071723e-03  3.5521942e-03 -1.3832499e-03
  2.2261145e-03 -1.0272069e-04  1.8701826e-03 -5.5740651e-04
 -4.0061306e-05  1.2200510e-03  3.9871671e-04 -5.1029399e-03
  5.8330186e-03  7.0035849e-03 -5.5042352e-03  2.8595570e-03
  7.9542620e-04 -3.5020120e-03  4.6632430e-03 -1.8540709e-03
 -2.389012

In [24]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"-!vQiIJHsZYCMg'lYFIJJeYEVlP Mgg'mgEyW-Z\nVOaIz\n?vKQIgq'xbFRXXJuooy-'QQZKRK&?&JjWqmQ\nRtPSz.hoEmPRNWVzF"

In [25]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [26]:
model.compile(optimizer='adam', loss=loss)

In [27]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [29]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) #this cell checks to see if tensorflow notices your gpu

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14266495093198202468
xla_global_id: -1
]


In [30]:
history = model.fit(data, epochs=5, callbacks=[checkpoint_callback]) #if tensorflow shows a gpu feel free to do upwards of 50 epochs
#if tf does not pick up you gpu, do less epochs to save time on computation

Epoch 1/5
172/172 [==============================] - 204s 1s/step - loss: 1.5685
Epoch 2/5
172/172 [==============================] - 206s 1s/step - loss: 1.4929
Epoch 3/5
172/172 [==============================] - 209s 1s/step - loss: 1.4387
Epoch 4/5
172/172 [==============================] - 211s 1s/step - loss: 1.3976
Epoch 5/5
172/172 [==============================] - 212s 1s/step - loss: 1.3630


In [31]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)#we are going to need to remake the model
#to accept 1 batch instead of 65 

In [32]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [38]:
checkpoint_num = 5
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

AttributeError: 'tensorflow.python.util._pywrap_checkpoint_reader.C' object has no attribute 'endswith'

In [35]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
    num_generate = 800

  # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
    text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
    temperature = 1.0

  # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
      # remove the batch dimension
    
        predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [36]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: romeo
romeob:
And noile viar, you murder'd for his crown,
Your life, you should do't: back we come too hund, and clemble impoder out
My soillour lords and fair and drops o' the wholence,
That's parton in clear me for this kingrs on, titlefore you
Bith's not sending 'bund, say noith.
But, by thy hand
That te shall bear him can do pilet!

ESCALUS:
Call my take I think AHORSUS:
I rvish; as you inpuritate thy founners, or every prince had beside
If stanmon here, when the contains my women lack
Where simple any feaster'd arms;
Or have you show I have no leads, we were they dr,
Wherefore, every more, the earthtand side
Than your lacues
In a hupbrance duty lay it please my breath,
Unto the great and on such a tomar in journing.

GREMIO:

ANGELO:
Bighty all that countern friends of her love,
That off ir, whi
